# Iterative Merging with Model Kinship
> Our Github [Merge Assistant Toolkit](https://github.com/zjunlp/ModelKinship).

> Open in Colab [This Notebook](https://colab.research.google.com/drive/141VAI89emgSIcwkswATEXSEENoAMywTO?usp=sharing)

This notebook demonstrates iterative merging using external tools to enhance large language model performance.
It leverages the following:

- [lm-evaluation-harness by EleutherAI](https://github.com/EleutherAI/lm-evaluation-harness) for providing a comprehensive evaluation framework for large language models.
- [mergekit by arcee-ai](https://github.com/arcee-ai/mergekit) for offering an essential toolkit for model merging experiments.

We express our gratitude to the contributors of these valuable tools.

In [ ]:
# @title ## Install requirements

# @markdown ### Run this section to install the required dependencies and log in to Hugging Face Hub.

!git clone https://github.com/EleutherAI/lm-evaluation-harness
!pip install -e ./lm-evaluation-harness
!git clone https://github.com/zjunlp/ModelKinship.git
!pip install -e ./ModelKinship
!git clone https://github.com/cg123/mergekit.git
!pip install -e ./mergekit
!pip install -qU huggingface_hub

username = "xxx" # @param {"type":"string"}
HF_Token = "hf_xxxx" # @param {"type":"string"}

from huggingface_hub import login
login(HF_Token)
api = HfApi(token=HF_Token)

In [ ]:
# @title ## Select Models

# @markdown ### Model 1
# @markdown Select your first model in huggingface hub.

Model_1 = "openchat/openchat-3.5-1210" # @param {"type":"string"}

# @markdown ### Model 1
# @markdown Select your second model in huggingface hub.

Model_2 = "meta-math/MetaMath-Mistral-7B" # @param {"type":"string"}

# @markdown ### Similarity metric for Model Kinship
# @markdown Select similarity metric for model kinship. Details can be found in our paper.

Similarity_metric = "Pearson Correlation Coefficient" # @param ["Pearson Correlation Coefficient","Cosine Similarity","Euclidean Distance"]
# Save config as yaml file
with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config)

cli = "merge_cal"
cli += " " + Model_1
cli += " " + Model_2


# Similarity Metrics
if Similarity_metric == "Euclidean Distance":
    cli += " ed"
elif Similarity_metric == "Cosine Similarity":
    cli += " cs"
elif Similarity_metric == "Pearson Correlation Coefficient"
    cli += " pcc"

print(cli)

# Calculation
!{cli}

In [ ]:
# @title ## Merge Models

# @markdown ### Name for the merged model
# @markdown Name your merged model.
Model_merge = "Merge_1_1" # @param {"type":"string"}

# @markdown ### Model 1 (base model)
# @markdown Select your first model in huggingface hub.

Model_1 = "openchat/openchat-3.5-1210" # @param {"type":"string"}

# @markdown ### Model 2
# @markdown Select your second model in huggingface hub.

Model_2 = "meta-math/MetaMath-Mistral-7B" # @param {"type":"string"}

config = f"""
slices:
  - sources:
      - model: {Model_1}
        layer_range: [0, 32]
      - model: {Model_2}
        layer_range: [0, 32]
merge_method: slerp
base_model: {Model_1}
parameters:
  t:
    - filter: self_attn
      value: [0, 0.5, 0.3, 0.7, 1]
    - filter: mlp
      value: [1, 0.5, 0.7, 0.3, 0]
    - value: 0.5
dtype: bfloat16
"""

with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(config)

!mergekit-yaml config.yaml merge --copy-tokenizer --allow-crimes --out-shard-size 1B --lazy-unpickle

In [ ]:
# @title ## Recycle Models

# @markdown ### Run this section to upload merged model.

from huggingface_hub import ModelCard, ModelCardData
from jinja2 import Template

template_text = """
---
license: apache-2.0
tags:
{%- for model in models %}
- {{ model }}
{%- endfor %}
---

{%- for model in models %}
* [{{ model }}](https://huggingface.co/{{ model }})
{%- endfor %}
"""

jinja_template = Template(template_text.strip())

data = yaml.safe_load(yaml_config)
models = [data["slices"][i]["sources"][0]["model"] for i in range(len(data["slices"]))]
content = jinja_template.render(
    model_name=Model_name,
    models=models,
    username=username,
)

# Save the model card
card = ModelCard(content)
card.save('merge/README.md')

api.create_repo(
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)
api.upload_folder(
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="merge",
)
